**BioBERT+CORAL**

In [ ]:
pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/Cleaned_Tweets_Standardized.xlsx"
df = pd.read_excel(file_path)


In [ ]:
df.head()

,Title,Text (Post),Label,Date
0,Facebook Group description,\nExercise regularly to control blood sugar\n\n,true,2023-12-13
1,NaN,Manage your carb intake reduces the risk of di...,true,NaT
2,NaN,Eat more fiber to reduce blood sugar levels,true,NaT
3,NaN,\nDrink water and stay hydrated helps to stay ...,true,NaT
4,How to live diabetes free life?,Lose extra weight. Losing weight reduces the r...,true,2023-12-18


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoConfig, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Ensure device compatibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ------------------------------
# Load and Process Dataset
# ------------------------------
file_path = "/mnt/data/Cleaned_Tweets.xlsx"
df = pd.read_excel(file_path)

# Drop missing values
df_cleaned = df.dropna(subset=["Text (Post)", "Label"]).copy()

# Convert labels to numerical values
label_mapping = {"true": 0, "false": 1, "exaggerated": 2, "misconstrued": 3}
df_cleaned.loc[:, "Label_ID"] = df_cleaned["Label"].map(label_mapping)

# Split into train, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df_cleaned["Text (Post)"], df_cleaned["Label_ID"], test_size=0.3, random_state=42, stratify=df_cleaned["Label_ID"]
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Define model name
model_name = "dmis-lab/biobert-base-cased-v1.1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ------------------------------
# Define Dataset Class
# ------------------------------
class MisinformationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create dataset instances
train_dataset = MisinformationDataset(train_texts, train_labels, tokenizer)
val_dataset = MisinformationDataset(val_texts, val_labels, tokenizer)
test_dataset = MisinformationDataset(test_texts, test_labels, tokenizer)

# ------------------------------
# Define CORAL Loss Function
# ------------------------------
def coral_loss(source_features, target_features):
    """CORAL loss function to align domain distributions."""
    source_mean = torch.mean(source_features, dim=0)
    target_mean = torch.mean(target_features, dim=0)

    source_cov = (source_features - source_mean).T @ (source_features - source_mean) / (source_features.shape[0] - 1)
    target_cov = (target_features - target_mean).T @ (target_features - target_mean) / (target_features.shape[0] - 1)

    loss = torch.norm(source_cov - target_cov, p='fro') + torch.norm(source_mean - target_mean, p=2)
    return loss

# ------------------------------
# Define BioBERT with CORAL
# ------------------------------
class BioBERT_Coral(nn.Module):
    def __init__(self, base_model_name, num_labels=4):
        super(BioBERT_Coral, self).__init__()
        self.config = AutoConfig.from_pretrained(base_model_name, num_labels=num_labels)
        self.feature_extractor = AutoModel.from_pretrained(base_model_name, config=self.config)
        self.classifier = nn.Linear(self.feature_extractor.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None, target_features=None):
        outputs = self.feature_extractor(input_ids=input_ids, attention_mask=attention_mask)

        # Ensure feature tensor is contiguous
        features = outputs.last_hidden_state[:, 0, :].contiguous()

        logits = self.classifier(features)

        loss = None
        if labels is not None:
            labels = labels.long()
            classification_loss = nn.CrossEntropyLoss()(logits, labels)

            if target_features is not None:
                target_features = target_features.contiguous()
                alignment_loss = coral_loss(features, target_features)
                loss = classification_loss + alignment_loss
            else:
                loss = classification_loss

        return {"loss": loss, "logits": logits}

# ------------------------------
# Load Model & Define Trainer
# ------------------------------
model = BioBERT_Coral(model_name, num_labels=len(set(label_mapping.values()))).to(device)

training_args = TrainingArguments(
    output_dir="./results_BioBERT_CORAL",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs_BioBERT_CORAL",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    save_total_limit=2,
    save_safetensors=False
)

# ------------------------------
# Define Metrics for Evaluation
# ------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1).numpy()
    labels = labels.numpy()
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ------------------------------
# Train Model
# ------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

# ------------------------------
# Save Trained Model
# ------------------------------
torch.save(model.state_dict(), "./content/drive/MyDrive/BioBERT_CORAL_model.pth")
tokenizer.save_pretrained("./content/drive/MyDrive/BioBERT_CORAL_model")

# ------------------------------
# Final Test Evaluation
# ------------------------------
print("Evaluating on Test Set...")
test_results = trainer.evaluate(test_dataset)
print(f"Final Evaluation Results on Test Set:\n{test_results}")


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'sympy.printing' has no attribute 'str'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U transformers


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess
formal_df = pd.read_csv("/content/drive/MyDrive/Diabetes_cleaned.csv").dropna(subset=["content", "label"])
informal_df = pd.read_csv("/content/drive/MyDrive/Corrected_Labeled.csv", encoding="ISO-8859-1").dropna(subset=["Text ", "Label"])
formal_df = formal_df.rename(columns={"content": "text", "label": "label"})
informal_df = informal_df.rename(columns={"Text ": "text", "Label": "label"})
formal_df["label"] = formal_df["label"].astype(int)
informal_df["label"] = informal_df["label"].astype(int)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_len)
        self.labels = labels.tolist()

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

# Split datasets
source_train, _ = train_test_split(formal_df, test_size=0.15, stratify=formal_df["label"], random_state=42)
target_train, target_val = train_test_split(informal_df, test_size=0.15, stratify=informal_df["label"], random_state=42)

train_dataset = TextDataset(source_train["text"], source_train["label"], tokenizer)
target_dataset = TextDataset(target_train["text"], target_train["label"], tokenizer)
val_dataset = TextDataset(target_val["text"], target_val["label"], tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
target_loader = DataLoader(target_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# CORAL loss
def coral_loss(source, target):
    d = source.size(1)
    source_mean = source.mean(0)
    target_mean = target.mean(0)
    source_cov = (source - source_mean).T @ (source - source_mean) / (source.size(0) - 1)
    target_cov = (target - target_mean).T @ (target - target_mean) / (target.size(0) - 1)
    return (torch.norm(source_cov - target_cov, p="fro")**2 + torch.norm(source_mean - target_mean, p=2)**2) / (4 * d**2)

# Model
class BioBERTWithCORAL(nn.Module):
    def __init__(self, base_model, num_labels=3):
        super().__init__()
        self.bert = AutoModel.from_pretrained(base_model)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        features = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(self.dropout(features))
        return logits, features

model = BioBERTWithCORAL("dmis-lab/biobert-base-cased-v1.1").to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
best_val_loss = float("inf")
patience = 2
patience_counter = 0

for epoch in range(10):
    model.train()
    total_loss = 0
    for batch, tgt in zip(train_loader, target_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits, features = model(input_ids, attention_mask)
        ce_loss = criterion(logits, labels)

        tgt_ids = tgt["input_ids"].to(device)
        tgt_mask = tgt["attention_mask"].to(device)
        with torch.no_grad():
            _, tgt_features = model(tgt_ids, tgt_mask)

        coral = coral_loss(features, tgt_features)
        loss = ce_loss + coral
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits, _ = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    print(f"→ Val Loss = {avg_val_loss:.4f}")
    print(classification_report(all_labels, all_preds, target_names=["True", "False", "Partially True"]))

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/BioBERT_CORAL_Final.pth")
        print("Best model saved")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break


Epoch 1: Train Loss = 40.9014
→ Val Loss = 1.2823
                precision    recall  f1-score   support

          True       0.70      0.34      0.46        87
         False       0.18      0.68      0.29        25
Partially True       0.00      0.00      0.00        24

      accuracy                           0.35       136
     macro avg       0.29      0.34      0.25       136
  weighted avg       0.48      0.35      0.35       136



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


✅ Best model saved
Epoch 2: Train Loss = 16.1715
→ Val Loss = 1.5638
                precision    recall  f1-score   support

          True       0.00      0.00      0.00        87
         False       0.19      1.00      0.31        25
Partially True       0.00      0.00      0.00        24

      accuracy                           0.18       136
     macro avg       0.06      0.33      0.10       136
  weighted avg       0.03      0.18      0.06       136



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 3: Train Loss = 9.5874
→ Val Loss = 1.7321
                precision    recall  f1-score   support

          True       0.75      0.03      0.07        87
         False       0.19      1.00      0.32        25
Partially True       0.00      0.00      0.00        24

      accuracy                           0.21       136
     macro avg       0.31      0.34      0.13       136
  weighted avg       0.51      0.21      0.10       136

⏹️ Early stopping triggered


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
